# Finding orthogonal equipartitions for sets in space with $8n$ points

Using integer precision in SageMath 9.5

## Code

In [1]:
import random
import itertools as it
from tqdm import tqdm

class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __add__(self, other):
        if isinstance(other, Vector):
            result_x = self.x + other.x
            result_y = self.y + other.y
            result_z = self.z + other.z
            return Vector(result_x, result_y, result_z)
        else:
            raise ValueError("Can only add two Vector objects together")

    def __sub__(self, other):
        if isinstance(other, Vector):
            result_x = self.x - other.x
            result_y = self.y - other.y
            result_z = self.z - other.z
            return Vector(result_x, result_y, result_z)
        else:
            raise ValueError("Can only subtract two Vector objects")

    def __mul__(self, scalar):
        result_x = self.x * scalar
        result_y = self.y * scalar
        result_z = self.z * scalar
        return Vector(result_x, result_y, result_z)

    def cross(self, other):
        result_x = self.y * other.z - self.z * other.y
        result_y = self.z * other.x - self.x * other.z
        result_z = self.x * other.y - self.y * other.x
        return Vector(result_x, result_y, result_z)

    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def norm2(self):
        return self.x * self.x + self.y * self.y + self.z * self.z
    
    def norm(self):
        return sqrt(self.norm2())
    
    def reduce(self):
        d = gcd(gcd(self.x,self.y),self.z)
        self.x,self.y,self.z = self.x/d,self.y/d,self.z/d

    def __str__(self):
        return f"({self.x}, {self.y}, {self.z})"

def points_to_vectors(L):
    """
    Converts a list of triples to a list of vectors
    """
    return [Vector(x,y,z) for x,y,z in L]
    
def generate_random_vectors(num_points=8, min_coord=-10, max_coord=10):
    """
    Generates a list of random 3D vectors
    """
    points = []
    for _ in range(num_points):
        x = random.randint(min_coord, max_coord)
        y = random.randint(min_coord, max_coord)
        z = random.randint(min_coord, max_coord)
        points.append(Vector(x, y, z))
    return points

def general_pos(vectors):
    """
    Checks if a set of vectors is in general position
    If not, prints indices of points defining planes
    """
    for a,b,c in it.combinations(vectors, 3):
        normal1 = (b-a).cross(c-a)
        normal1.reduce()
        N1,Z1,P1 = split_by_point_normal(a,normal1,vectors)
        if len(Z1)!=3:
            print(Z1)
            return False
        for d,e in it.combinations(vectors, 2):
            normal2 = normal1.cross(e-d)
            normal2.reduce()
            N2,Z2,P2 = split_by_point_normal(d,normal2, vectors)
            if len(Z2)!=2:
                print(Z1,Z2)
                return False
            normal3 = normal1.cross(normal2)
            normal3.reduce()
            for f in vectors:
                N3,Z3,P3 = split_by_point_normal(f,normal3, vectors)
                if len(Z3)!=1:
                    print(Z1,Z2,Z3)
                    return False
    return True

def split_by_point_normal(point, normal, vectors):
    """
    Partitions vectors by the plane normal to normal through point
    Returns two lists of indices
    """
    nn2 = normal.norm2()
    assert nn2>0, f"Norm 0"
    Positive = []
    Zero = []
    Negative = []
    for i,v in enumerate(vectors):
        ndv = normal.dot(v) - normal.dot(point)
        if ndv*ndv == 0:
            Zero.append(i)
        elif ndv > 0:
            Positive.append(i)
        else:
            Negative.append(i)
    return Negative, Zero, Positive

def make_choices(N,Z,P):
    """
    Takes elments from Z and adds them to N and P so that they have the same size
    """
    assert (len(P)+len(Z)+len(N))%2==0, "Need an even number of total elements"
    if len(N)+len(Z)<len(P) or len(P)+len(Z)<len(N): #Not possible to distribute evenly
        return
    num_negs = (len(P)+len(Z)-len(N))//2 #Number of points that should be added to N
    for new_negs in it.combinations(Z,num_negs):
        newN=N.copy()
        newP=P.copy()
        for i in Z:
            if i in new_negs:
                newN.append(i)
            else:
                newP.append(i)
        yield newN,newP
        
def print_vectors(vectors,as_list=False):
    """
    Prints a list of vectors in a readable form, if as_list==True it prints them as a python list
    """
    if as_list:
        res='['
        for v in vectors:
            res += f'{v}, '
        print(res[:-2]+']')
    else:
        for i,v in enumerate(vectors):
            print(f'{i}: {v}')

def print_partition(partition):
    """
    Prints a partition in a readable form
    """
    for N,P,x,n,Z in partition:
        print(f'Plane defined by {Z},')
        print(f'orthogonal to {n} through {x} separates')
        print(f'{N} and {P}')

def find_321_partition(vectors,stop_at_first=True):
    """
    Finds a partition of type 3,2,1
    """
    Equipartitions = []
    for a,b,c in it.combinations(vectors, 3):
        normal1 = (b-a).cross(c-a)
        normal1.reduce()
        N1,Z1,P1 = split_by_point_normal(a,normal1,vectors)
        assert len(Z1)==3, f"Points not in general possition: {Z1}"
        for newN1,newP1 in make_choices(N1,Z1,P1):
            #newP1 and newN1 have the same size
            for d,e in it.combinations(vectors, 2):
                normal2 = normal1.cross(e-d)
                normal2.reduce()
                N2,Z2,P2 = split_by_point_normal(d,normal2, vectors)
                assert len(Z2)==2, f"Points not in general possition: {Z1}, {Z2}"
                for newN2,newP2 in make_choices(N2,Z2,P2):
                    #newP2 and newN2 have the same size
                    #Check if they split into 4 equal parts
                    cuadrant1 = {v for v in newN1 if v in newN2}
                    if len(cuadrant1)!=len(vectors)//4:
                        continue
                    cuadrant2 = {v for v in newN1 if v not in newN2}
                    cuadrant3 = {v for v in newP1 if v in newN2}
                    normal3 = normal1.cross(normal2)
                    normal3.reduce()
                    for f in vectors:
                        N3,Z3,P3 = split_by_point_normal(f,normal3, vectors)
                        assert len(Z3)==1, f"Points not in general possition: {Z1}, {Z2}, {Z3}"
                        for newN3,newP3 in make_choices(N3,Z3,P3):
                            #Check if they split into 8 equal parts
                            if len(cuadrant1.intersection(newN3))!=len(vectors)//8 or len(cuadrant2.intersection(newN3))!=len(vectors)//8 or len(cuadrant3.intersection(newN3))!=len(vectors)//8:
                                continue
                            p = ((newN1,newP1,a,normal1,Z1),(newN2,newP2,d,normal2,Z2),(newN3,newP3,f,normal3,Z3))
                            Equipartitions.append(p)
                            if stop_at_first:
                                return Equipartitions
    return Equipartitions

def compute_normals(u1,u2,u3):
    """
    For partitions of type 2,2,2, given vectors u1,u2,u3 searches for orthogonal vectors v1,v2,v3 satisfying that vi is ortogonal to ui
    Returns 0 if none is found
    """
    v11=u1.cross(Vector(1,0,0));v12=u1.cross(Vector(0,1,0));
    v21=u2.cross(Vector(1,0,0));v22=u2.cross(Vector(0,1,0));
    v31=u3.cross(Vector(1,0,0));v32=u3.cross(Vector(0,1,0));
    a1 = v21.dot(v31); b1 = v21.dot(v32); c1 = v22.dot(v31); d1 = v22.dot(v32)
    a2 = v31.dot(v11); b2 = v31.dot(v12); c2 = v32.dot(v11); d2 = v32.dot(v12)
    a3 = v11.dot(v21); b3 = v11.dot(v22); c3 = v12.dot(v21); d3 = v12.dot(v22)
    
    disc = (b1*b2*b3 + c1*c2*c3 - a3*b2*d1 - a2*c3*d1 - a1*b3*d2 + a3*c1*d2 + a2*b1*d3 - a1*c2*d3)**2 - 4*(a2*b1*b3 - a1*b3*c2 + a3*c1*c2 - a2*a3*d1)*(-(b2*c3*d1) + c1*c3*d2 + b1*b2*d3 - a1*d2*d3)
    if disc < 0:
        return 0,0,0
    root = sqrt(disc)
    den1 = 2*(a2*b1*b3 - a1*b3*c2 + a3*c1*c2 - a2*a3*d1)
    den2 = 2*(a3*b2*b1 - a2*b1*c3 + a1*c2*c3 - a3*a1*d2)
    den3 = 2*(a1*b3*b2 - a3*b2*c1 + a2*c3*c1 - a1*a2*d3)
    assert den1!=0 and den2!=0 and den3!=0, f"Not in general position {den1,den2,den3}"
    
    x1 = (-(b1*b2*b3) - c1*c2*c3 + a3*b2*d1 + a2*c3*d1 + a1*b3*d2 - a3*c1*d2 - a2*b1*d3 + a1*c2*d3 + root)/den1
    x2 = (-(b1*b2*b3) - c1*c2*c3 - a3*b2*d1 + a2*c3*d1 + a1*b3*d2 + a3*c1*d2 + a2*b1*d3 - a1*c2*d3 + root)/den2
    x3 = (-(b1*b2*b3) - c1*c2*c3 + a3*b2*d1 - a2*c3*d1 - a1*b3*d2 + a3*c1*d2 + a2*b1*d3 + a1*c2*d3 + root)/den3

    return v11*x1 + v12, v21*x2 + v22, v31*x3 + v32

def find_222_partition(vectors,stop_at_first=True):
    """
    Finds a partition of type 2,2,2
    """
    Equipartitions = []
    for (a1,b1),(a2,b2),(a3,b3) in it.combinations(it.combinations(vectors, 2),3):
        normal1,normal2,normal3 = compute_normals(b1-a1,b2-a2,b3-a3)
        if normal1 == 0:
            #No partition exists
            continue
        N1,Z1,P1 = split_by_point_normal(a1,normal1,vectors)
        N2,Z2,P2 = split_by_point_normal(a2,normal2,vectors)
        N3,Z3,P3 = split_by_point_normal(a3,normal3,vectors)
        assert len(Z1)==2 and len(Z2)==2 and len(Z3)==2, f"Points not in general possition: {Z1}, {Z2}, {Z3}"
        for newN1,newP1 in make_choices(N1,Z1,P1):
            for newN2,newP2 in make_choices(N2,Z2,P2):
                #Check if they split into 4 equal parts
                cuadrant1 = {v for v in newN1 if v in newN2}
                if len(cuadrant1)!=len(vectors)//4:
                    continue
                cuadrant2 = {v for v in newN1 if v not in newN2}
                cuadrant3 = {v for v in newP1 if v in newN2}
                for newN3,newP3 in make_choices(N3,Z3,P3):
                    #Check if they split into 8 equal parts
                    if len(cuadrant1.intersection(newN3))!=len(vectors)//8 or len(cuadrant2.intersection(newN3))!=len(vectors)//8 or len(cuadrant3.intersection(newN3))!=len(vectors)//8:
                        continue
                    p = ((newN1,newP1,a,normal1,Z1),(newN2,newP2,d,normal2,Z2),(newN3,newP3,f,normal3,Z3))
                    Equipartitions.append(p)
                    if stop_at_first:
                        return Equipartitions
    return Equipartitions

def find_partition(vectors,stop_at_first=True):
    """
    Searches for an orthogonal equipartition 
    """
    Equipartitions = find_321_partition(vectors,stop_at_first=stop_at_first)
    if len(Equipartitions)>0 and stop_at_first:
        return Equipartitions
    Equipartitions += find_222_partition(vectors,stop_at_first=stop_at_first)
    return Equipartitions

## Random examples

Looks for orthogonal equipartitions in sets of $8$ random points chosen uniformly and independently in a cube. Saves and counts the examples without equipartitions.

The general position is not checked thoroughly, only when needed. When an equipartiton is found the program continues no the next set, so not all sets where an equipartition is found are necessarily in general position.

In [2]:
num_points=8
N=10000

NumExamples = 0
ExamplesNoEquipart8 = []
for TotalTries in tqdm(range(N)):
    vectors = generate_random_vectors(num_points=num_points, min_coord=-100, max_coord=100)
    try:
        partition = find_partition(vectors)
        NumExamples += 1
        if len(partition)==0:
            ExamplesNoEquipart8.append(vectors)
    except Exception as error:
        #If they are not in general position
        continue
print("Sets found in general position:",NumExamples)
print("Sets without equipartition:    ",len(ExamplesNoEquipart8))
print("Proportion:                    ",1.*len(ExamplesNoEquipart8)/NumExamples)
#for vectors in ExamplesNoEquipart8:
#    print_vectors(vectors,as_list=True)

100%|█████████████████████████████████████████████████████████████████████████████| 10000/10000 [10:10<00:00, 16.39it/s]

Sets found in general position: 9975
Sets without equipartition:     10
Proportion:                     0.00100250626566416


In [3]:
vectors = ExamplesNoEquipart8[0]
if not general_pos(vectors):
    print("Not in general position!")
print_vectors(vectors)

0: (-68, -84, -18)
1: (82, -12, -4)
2: (-81, -94, 36)
3: (38, 37, -9)
4: (26, 90, 8)
5: (-67, -49, 90)
6: (11, 73, 33)
7: (-95, -94, 87)


## Examples in the moment curve

### Examples with no orthogonal equipartitions

In [4]:
vectors = [Vector(x,x*x,x**3) for x in range(8)]
print_vectors(vectors)
if not general_pos(vectors):
    print("Not in general position!")
partitions = find_partition(vectors)
partitions

0: (0, 0, 0)
1: (1, 1, 1)
2: (2, 4, 8)
3: (3, 9, 27)
4: (4, 16, 64)
5: (5, 25, 125)
6: (6, 36, 216)
7: (7, 49, 343)


[]

In [5]:
vectors = [Vector(x,x*x,x**3) for x in range(-7,8,2)]
print_vectors(vectors)
if not general_pos(vectors):
    print("Not in general position!")
partitions = find_partition(vectors)
partitions

0: (-7, 49, -343)
1: (-5, 25, -125)
2: (-3, 9, -27)
3: (-1, 1, -1)
4: (1, 1, 1)
5: (3, 9, 27)
6: (5, 25, 125)
7: (7, 49, 343)


[]

### Examples with orthogonal equipartitions

In [6]:
vectors = [Vector(x,x*x,x**3) for x in [-4, -3, -2, -1, 4, 5, 6, 7]]
print_vectors(vectors)
if not general_pos(vectors):
    print("Not in general position!")
partitions = find_partition(vectors,stop_at_first=False)
print_partition(partitions[0])

0: (-4, 16, -64)
1: (-3, 9, -27)
2: (-2, 4, -8)
3: (-1, 1, -1)
4: (4, 16, 64)
5: (5, 25, 125)
6: (6, 36, 216)
7: (7, 49, 343)
Plane defined by [0, 2, 5],
orthogonal to (-22, 1, 1) through (-4, 16, -64) separates
[3, 4, 0, 5] and [1, 6, 7, 2]
Plane defined by [4, 6],
orthogonal to (66, 1673, -221) through (4, 16, 64) separates
[2, 3, 7, 4] and [0, 1, 5, 6]
Plane defined by [3],
orthogonal to (-947, -2398, -18436) through (-1, 1, -1) separates
[4, 5, 6, 7] and [0, 1, 2, 3]


In [7]:
vectors = [Vector(x,x*x,x**3) for x in [-7, -6, -5, -4, 1, 2, 3, 4]]
print_vectors(vectors)
if not general_pos(vectors):
    print("Not in general position!")
partitions = find_partition(vectors,stop_at_first=False)
print_partition(partitions[0])

0: (-7, 49, -343)
1: (-6, 36, -216)
2: (-5, 25, -125)
3: (-4, 16, -64)
4: (1, 1, 1)
5: (2, 4, 8)
6: (3, 9, 27)
7: (4, 16, 64)
Plane defined by [1, 4, 6],
orthogonal to (-21, 2, 1) through (-6, 36, -216) separates
[0, 5, 1, 6] and [2, 3, 7, 4]
Plane defined by [0, 2],
orthogonal to (23, 229, 25) through (-7, 49, -343) separates
[3, 4, 5, 0] and [1, 6, 7, 2]
Plane defined by [3],
orthogonal to (-179, 548, -4855) through (-4, 16, -64) separates
[4, 5, 6, 7] and [0, 1, 2, 3]
